In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import regionate as reg
import sectionate as sec

In [3]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [4]:
import pickle

## Load basin boundaries as a `regionate.Region` consisting of 35 instances of `regionate.Basin`

In [5]:
import scipy.io
basins = scipy.io.loadmat('boundaries/PJ2010_abyssal_basin_boundaries.mat', simplify_cells=True, squeeze_me=True)['bsn_only']
region = reg.Region({
    str(basin['bsn_num'].replace("_", "")):
    reg.Basin(basin['bsn'][0], basin['bsn'][1], str(basin['bsn_num'].replace("_", "")), counterclockwise=True)
    for basin in basins
})
    
# Slightly shift coordinates off of supergrid nodes to avoid rounding errors
for bname, b in region.Basins.items():
    b.lons -= 1.e-3
    b.lats -= 1.e-3

## Conform regions to CM4p25 grid

In [6]:
region_gridded = region.copy()

import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")

    # get correct geo coords from super grid
    sg = xr.open_dataset("/archive/Raphael.Dussin/datasets/OM4p25/c192_OM4_025_grid_No_mg_drag_v20160808_unpacked/ocean_hgrid.nc")
    zdiag_path = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p25_v7_allfixes/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/"
    ocean_grid = xr.open_dataset(f"{zdiag_path}ocean_annual_z.static.nc")
    ocean_grid = ocean_grid.assign_coords({'geolon'  : xr.DataArray(sg['x'][1::2,1::2].data, dims=["yh", "xh"]),
                                           'geolat'  : xr.DataArray(sg['y'][1::2,1::2].data, dims=["yh", "xh"]),
                                           'geolon_u': xr.DataArray(sg['x'][1::2,0::2].data, dims=["yh", "xq"]),
                                           'geolat_u': xr.DataArray(sg['y'][1::2,0::2].data, dims=["yh", "xq"]),
                                           'geolon_v': xr.DataArray(sg['x'][0::2,1::2].data, dims=["yq", "xh"]),
                                           'geolat_v': xr.DataArray(sg['y'][0::2,1::2].data, dims=["yq", "xh"]),
                                           'geolon_c': xr.DataArray(sg['x'][0::2,0::2].data, dims=["yq", "xq"]),
                                           'geolat_c': xr.DataArray(sg['y'][0::2,0::2].data, dims=["yq", "xq"])})

# Conform basin boundaries to model grid
print("Conforming regions to ocean grid: ", end="")
for (bname, b) in region_gridded.Basins.items():
    print(bname, end=", ")
    reg.conform_basin_to_ocean_grid(b, ocean_grid)
reg.check_global_coverage(region_gridded)
region_gridded.find_all_overlaps(face_indices=True)

# `pickle` boundary indices and area mask
regionFile_CM4p25 = "/work/hfd/datasets/regionate/pickled_regions/PJ2010_region_gridded_CM4p25"
with open(regionFile_CM4p25, 'wb') as pickle_file:
    pickle.dump(region_gridded, pickle_file)

Conforming regions to ocean grid: 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 1, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 2, 30, 31, 32, 33, 3, 4, 5, 6, 7, 8, 9, 35, 34, 

## Conform regions to CM4p125 grid

In [7]:
region_gridded = region.copy()

# Load CMp125 grid
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    
    # get correct geo coords from super grid
    sg = xr.open_dataset("/archive/Raphael.Dussin/datasets/OM4p125/mosaic_c192_om4p125_bedmachine_v20210310_hydrographyKDunne20210614_unpacked/ocean_hgrid.nc")
    zdiag_path = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210706/CM4_piControl_c192_OM4p125_v7/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/"
    ocean_grid = xr.open_dataset(f"{zdiag_path}ocean_annual_z.static.nc")
    ocean_grid = ocean_grid.assign_coords({'geolon'  : xr.DataArray(sg['x'][1::2,1::2].data, dims=["yh", "xh"]),
                                           'geolat'  : xr.DataArray(sg['y'][1::2,1::2].data, dims=["yh", "xh"]),
                                           'geolon_u': xr.DataArray(sg['x'][1::2,0::2].data, dims=["yh", "xq"]),
                                           'geolat_u': xr.DataArray(sg['y'][1::2,0::2].data, dims=["yh", "xq"]),
                                           'geolon_v': xr.DataArray(sg['x'][0::2,1::2].data, dims=["yq", "xh"]),
                                           'geolat_v': xr.DataArray(sg['y'][0::2,1::2].data, dims=["yq", "xh"]),
                                           'geolon_c': xr.DataArray(sg['x'][0::2,0::2].data, dims=["yq", "xq"]),
                                           'geolat_c': xr.DataArray(sg['y'][0::2,0::2].data, dims=["yq", "xq"])})

# Conform basin boundaries to model grid
print("Conforming regions to ocean grid: ", end="")
for (bname, b) in region_gridded.Basins.items():
    print(bname, end=", ")
    reg.conform_basin_to_ocean_grid(b, ocean_grid)
reg.check_global_coverage(region_gridded)
region_gridded.find_all_overlaps(face_indices=True)

# `pickle` boundary indices and area mask
regionFile_CM4p125 = "/work/hfd/datasets/regionate/pickled_regions/PJ2010_region_gridded_CM4p125"
with open(regionFile_CM4p125, 'wb') as pickle_file:
    pickle.dump(region_gridded, pickle_file)

Conforming regions to ocean grid: 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 1, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 2, 30, 31, 32, 33, 3, 4, 5, 6, 7, 8, 9, 35, 34, 